In [4]:
struct x_values{
    double xpos[2];
    double xneg[2];        
};

In [5]:
x_values isocrony_x(Double_t* params_1, const Double_t* err_params_1, Double_t* params_2, const Double_t* err_params_2){
    x_values x;

    // non usare valori assoluti per a, b e c per evitare di ottenere valore di isocronia errato.
    double a = params_1[0]-params_2[0], err_a = abs(err_params_1[0])+abs(err_params_2[0]);
    double b = params_1[1]-params_2[1], err_b = abs(err_params_1[1])+abs(err_params_2[1]);
    double c = params_1[2]-params_2[2], err_c = abs(err_params_1[2])+abs(err_params_2[2]);

    double delta = pow(b, 2) - 4*a*c;

    if(delta<0){
        std::cout << "delta is negative!" << std::endl;
        return {{0,0}, {0,0}};
    }

    x.xpos[0] = (-b+sqrt(delta))/(2*a);
    x.xneg[0] = (-b-sqrt(delta))/(2*a);

    double dda_sqr_pos = pow(((-1)*c/(a*sqrt(delta))) - ((-b + sqrt(delta))/(2*a*a)), 2);
    double ddb_sqr_pos = pow((-1 + b/sqrt(delta))/(2*a), 2);
    double ddc_sqr_pos = pow(-1/sqrt(delta), 2);

    x.xpos[1] = sqrt((dda_sqr_pos*err_a*err_a) + (ddb_sqr_pos*err_b*err_b) + (ddc_sqr_pos*err_c*err_c));

    double dda_sqr_neg = pow((b*b - 2*a*c + b*sqrt(delta))/(2*a*a*sqrt(delta)), 2);
    double ddb_sqr_neg = pow((-1 - (b/sqrt(delta)))/(2*a), 2);
    double ddc_sqr_neg = pow(1/sqrt(delta), 2);

    x.xneg[1] = sqrt((dda_sqr_neg*err_a*err_a) + (ddb_sqr_neg*err_b*err_b) + (ddc_sqr_neg*err_c*err_c));

    return x;
}

In [7]:
double* get_isoX(x_values x, TGraphErrors _g){
    
    double* _x = new double[2];

    if(x.xpos[0]>TMath::MinElement(_g.GetN(), _g.GetX())){
        _x[0] = x.xpos[0]; _x[1] = x.xpos[1];
    }else{
        _x[0] = x.xneg[0]; _x[1] = x.xneg[1];
    }
    
    return _x;

}

In [8]:
double get_err_T(double x_iso, TGraphErrors g1, TGraphErrors g2){

    double Terr_iso_1 = 0;
    double Terr_iso_2 = 0;
    double range = abs(g1.GetX()[0]-x_iso);

    for(int i=0; i<g1.GetN(); i++){
        // std::cout << i << " " << abs(x_iso-g1.GetX()[i]) << " x: " << g1.GetX()[i] << std::endl;
        // algorimo per identificazione dei valori di T1 T2 più
        // vicini al valore di isocronia

        if((abs(x_iso-g1.GetX()[i])<range)){
            Terr_iso_1 = g1.GetErrorY(i);
            Terr_iso_2 = g2.GetErrorY(i);
            range = abs(x_iso-g1.GetX()[i]);
        }
    }

    return sqrt(pow(Terr_iso_1, 2)+pow(Terr_iso_2, 2));
}